In [ ]:
# 파이썬 버전
import sys
print(sys.version)

3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]


In [ ]:
# OS 버전
import platform 
platform.platform()

'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic'

# 0. Import

In [ ]:
!pip install optuna

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import optuna
from optuna import Trial, visualization
from sklearn.metrics import mean_absolute_error

import xgboost as xgb

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, plot_confusion_matrix
from sklearn.ensemble import ExtraTreesRegressor
import matplotlib.pyplot as plt
import pickle
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.ensemble import GradientBoostingClassifier as GBC

import lightgbm as lgb

!pip install workalendar
from workalendar.asia import SouthKorea

# 1. Data load

외부 데이터 출처
*아래 데이터를 기반으로 데이터 형태만 바꿔서 저장한 후 사용
- 전산업생산지수: KOSIS 국가통계포털 전산업생산지수(원지수)
https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1JH20151&vw_cd=MT_ZTITLE&list_id=J1_10&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_ZTITLE 

- 가스기름 상대가격: 한국가스공사_도시가스 산업용 월별 상대가격지수(산업에서 가스의 대체제는 기름(석유)이므로)
https://www.data.go.kr/data/15066079/fileData.do

- 가스전기 상대가격: 한국가스공사_도시가스 민수용 월별 상대가격지수(민간에서 가스의 대체제는 전기이므로)
https://www.data.go.kr/data/15066075/fileData.do
- temp2013, temp2014, temp2015, temp2016, temp2017, temp2018 : 2013~2018년 기온 데이터 (https://data.kma.go.kr/cmmn/main.do)

In [ ]:
path = '/content/drive/MyDrive/가스공급량 수요예측 모델개발'

In [ ]:
total = pd.read_csv(path+'/한국가스공사_시간별 공급량_20181231.csv', encoding='cp949')

In [ ]:
GO_price = pd.read_csv(path+'/가스기름 상대가격(월별).csv')
GE_price = pd.read_csv(path+'/가스전기 상대가격(월별).csv')
IAIP = pd.read_csv(path+'/전산업생산지수(월별).csv')
gas_d = pd.read_csv(path+'/한국가스공사_도시가스 산업용 월별 가스수요산업지수_20210630.csv', encoding='cp949')

In [ ]:
weather_2013 = pd.read_csv(path+'/temp2013.csv', encoding='cp949')
weather_2014 = pd.read_csv(path+'/temp2014.csv', encoding='cp949')
weather_2015 = pd.read_csv(path+'/temp2015.csv', encoding='cp949')
weather_2016 = pd.read_csv(path+'/temp2016.csv', encoding='cp949')
weather_2017 = pd.read_csv(path+'/temp2017.csv', encoding='cp949')
weather_2018 = pd.read_csv(path+'/temp2018.csv', encoding='cp949')

# 2. 데이터 전처리

In [ ]:
total['구분'].unique()

array(['A', 'B', 'C', 'D', 'E', 'G', 'H'], dtype=object)

In [ ]:
d_map = {}
for i, d in enumerate(total['구분'].unique()):
    d_map[d] = i
total['구분'] = total['구분'].map(d_map)

In [ ]:
total['연월일'] = pd.to_datetime(total['연월일'])

In [ ]:
total['year'] = total['연월일'].dt.year
total['month'] = total['연월일'].dt.month
total['day'] = total['연월일'].dt.day
total['weekday'] = total['연월일'].dt.weekday

## 구분별/월별 이상치 처리

In [ ]:
def outliers_iqr(df):
    quartile_1, quartile_3 = np.percentile(df, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return lower_bound, upper_bound

*   구분별로 이상치를 처리하기 위해 구분별 데이터 셋 분리


In [ ]:
total_0 = total[total['구분'] == 0]
total_1 = total[total['구분'] == 1]
total_2 = total[total['구분'] == 2]
total_3 = total[total['구분'] == 3]
total_4 = total[total['구분'] == 4]
total_5 = total[total['구분'] == 5]
total_6 = total[total['구분'] == 6]

*   구분별/월별로 Q1, Q3 값 저장

In [ ]:
lower_0_1, upper_0_1 = outliers_iqr(total_0[total_0['month'] == 1]['공급량'])
lower_0_2, upper_0_2 = outliers_iqr(total_0[total_0['month'] == 2]['공급량'])
lower_0_3, upper_0_3 = outliers_iqr(total_0[total_0['month'] == 3]['공급량'])
lower_0_4, upper_0_4 = outliers_iqr(total_0[total_0['month'] == 4]['공급량'])
lower_0_5, upper_0_5 = outliers_iqr(total_0[total_0['month'] == 5]['공급량'])
lower_0_6, upper_0_6 = outliers_iqr(total_0[total_0['month'] == 6]['공급량'])
lower_0_7, upper_0_7 = outliers_iqr(total_0[total_0['month'] == 7]['공급량'])
lower_0_8, upper_0_8 = outliers_iqr(total_0[total_0['month'] == 8]['공급량'])
lower_0_9, upper_0_9 = outliers_iqr(total_0[total_0['month'] == 9]['공급량'])
lower_0_10, upper_0_10 = outliers_iqr(total_0[total_0['month'] == 10]['공급량'])
lower_0_11, upper_0_11 = outliers_iqr(total_0[total_0['month'] == 11]['공급량'])
lower_0_12, upper_0_12 = outliers_iqr(total_0[total_0['month'] == 12]['공급량'])

In [ ]:
lower_1_1, upper_1_1 = outliers_iqr(total_1[total_1['month'] == 1]['공급량'])
lower_1_2, upper_1_2 = outliers_iqr(total_1[total_1['month'] == 2]['공급량'])
lower_1_3, upper_1_3 = outliers_iqr(total_1[total_1['month'] == 3]['공급량'])
lower_1_4, upper_1_4 = outliers_iqr(total_1[total_1['month'] == 4]['공급량'])
lower_1_5, upper_1_5 = outliers_iqr(total_1[total_1['month'] == 5]['공급량'])
lower_1_6, upper_1_6 = outliers_iqr(total_1[total_1['month'] == 6]['공급량'])
lower_1_7, upper_1_7 = outliers_iqr(total_1[total_1['month'] == 7]['공급량'])
lower_1_8, upper_1_8 = outliers_iqr(total_1[total_1['month'] == 8]['공급량'])
lower_1_9, upper_1_9 = outliers_iqr(total_1[total_1['month'] == 9]['공급량'])
lower_1_10, upper_1_10 = outliers_iqr(total_1[total_1['month'] == 10]['공급량'])
lower_1_11, upper_1_11 = outliers_iqr(total_1[total_1['month'] == 11]['공급량'])
lower_1_12, upper_1_12 = outliers_iqr(total_1[total_1['month'] == 12]['공급량'])

In [ ]:
lower_2_1, upper_2_1 = outliers_iqr(total_2[total_2['month'] == 1]['공급량'])
lower_2_2, upper_2_2 = outliers_iqr(total_2[total_2['month'] == 2]['공급량'])
lower_2_3, upper_2_3 = outliers_iqr(total_2[total_2['month'] == 3]['공급량'])
lower_2_4, upper_2_4 = outliers_iqr(total_2[total_2['month'] == 4]['공급량'])
lower_2_5, upper_2_5 = outliers_iqr(total_2[total_2['month'] == 5]['공급량'])
lower_2_6, upper_2_6 = outliers_iqr(total_2[total_2['month'] == 6]['공급량'])
lower_2_7, upper_2_7 = outliers_iqr(total_2[total_2['month'] == 7]['공급량'])
lower_2_8, upper_2_8 = outliers_iqr(total_2[total_2['month'] == 8]['공급량'])
lower_2_9, upper_2_9 = outliers_iqr(total_2[total_2['month'] == 9]['공급량'])
lower_2_10, upper_2_10 = outliers_iqr(total_2[total_2['month'] == 10]['공급량'])
lower_2_11, upper_2_11 = outliers_iqr(total_2[total_2['month'] == 11]['공급량'])
lower_2_12, upper_2_12 = outliers_iqr(total_2[total_2['month'] == 12]['공급량'])

In [ ]:
lower_3_1, upper_3_1 = outliers_iqr(total_3[total_3['month'] == 1]['공급량'])
lower_3_2, upper_3_2 = outliers_iqr(total_3[total_3['month'] == 2]['공급량'])
lower_3_3, upper_3_3 = outliers_iqr(total_3[total_3['month'] == 3]['공급량'])
lower_3_4, upper_3_4 = outliers_iqr(total_3[total_3['month'] == 4]['공급량'])
lower_3_5, upper_3_5 = outliers_iqr(total_3[total_3['month'] == 5]['공급량'])
lower_3_6, upper_3_6 = outliers_iqr(total_3[total_3['month'] == 6]['공급량'])
lower_3_7, upper_3_7 = outliers_iqr(total_3[total_3['month'] == 7]['공급량'])
lower_3_8, upper_3_8 = outliers_iqr(total_3[total_3['month'] == 8]['공급량'])
lower_3_9, upper_3_9 = outliers_iqr(total_3[total_3['month'] == 9]['공급량'])
lower_3_10, upper_3_10 = outliers_iqr(total_3[total_3['month'] == 10]['공급량'])
lower_3_11, upper_3_11 = outliers_iqr(total_3[total_3['month'] == 11]['공급량'])
lower_3_12, upper_3_12 = outliers_iqr(total_3[total_3['month'] == 12]['공급량'])

In [ ]:
lower_4_1, upper_4_1 = outliers_iqr(total_4[total_4['month'] == 1]['공급량'])
lower_4_2, upper_4_2 = outliers_iqr(total_4[total_4['month'] == 2]['공급량'])
lower_4_3, upper_4_3 = outliers_iqr(total_4[total_4['month'] == 3]['공급량'])
lower_4_4, upper_4_4 = outliers_iqr(total_4[total_4['month'] == 4]['공급량'])
lower_4_5, upper_4_5 = outliers_iqr(total_4[total_4['month'] == 5]['공급량'])
lower_4_6, upper_4_6 = outliers_iqr(total_4[total_4['month'] == 6]['공급량'])
lower_4_7, upper_4_7 = outliers_iqr(total_4[total_4['month'] == 7]['공급량'])
lower_4_8, upper_4_8 = outliers_iqr(total_4[total_4['month'] == 8]['공급량'])
lower_4_9, upper_4_9 = outliers_iqr(total_4[total_4['month'] == 9]['공급량'])
lower_4_10, upper_4_10 = outliers_iqr(total_4[total_4['month'] == 10]['공급량'])
lower_4_11, upper_4_11 = outliers_iqr(total_4[total_4['month'] == 11]['공급량'])
lower_4_12, upper_4_12 = outliers_iqr(total_4[total_4['month'] == 12]['공급량'])

In [ ]:
lower_5_1, upper_5_1 = outliers_iqr(total_5[total_5['month'] == 1]['공급량'])
lower_5_2, upper_5_2 = outliers_iqr(total_5[total_5['month'] == 2]['공급량'])
lower_5_3, upper_5_3 = outliers_iqr(total_5[total_5['month'] == 3]['공급량'])
lower_5_4, upper_5_4 = outliers_iqr(total_5[total_5['month'] == 4]['공급량'])
lower_5_5, upper_5_5 = outliers_iqr(total_5[total_5['month'] == 5]['공급량'])
lower_5_6, upper_5_6 = outliers_iqr(total_5[total_5['month'] == 6]['공급량'])
lower_5_7, upper_5_7 = outliers_iqr(total_5[total_5['month'] == 7]['공급량'])
lower_5_8, upper_5_8 = outliers_iqr(total_5[total_5['month'] == 8]['공급량'])
lower_5_9, upper_5_9 = outliers_iqr(total_5[total_5['month'] == 9]['공급량'])
lower_5_10, upper_5_10 = outliers_iqr(total_5[total_5['month'] == 10]['공급량'])
lower_5_11, upper_5_11 = outliers_iqr(total_5[total_5['month'] == 11]['공급량'])
lower_5_12, upper_5_12 = outliers_iqr(total_5[total_5['month'] == 12]['공급량'])

In [ ]:
lower_6_1, upper_6_1 = outliers_iqr(total_6[total_6['month'] == 1]['공급량'])
lower_6_2, upper_6_2 = outliers_iqr(total_6[total_6['month'] == 2]['공급량'])
lower_6_3, upper_6_3 = outliers_iqr(total_6[total_6['month'] == 3]['공급량'])
lower_6_4, upper_6_4 = outliers_iqr(total_6[total_6['month'] == 4]['공급량'])
lower_6_5, upper_6_5 = outliers_iqr(total_6[total_6['month'] == 5]['공급량'])
lower_6_6, upper_6_6 = outliers_iqr(total_6[total_6['month'] == 6]['공급량'])
lower_6_7, upper_6_7 = outliers_iqr(total_6[total_6['month'] == 7]['공급량'])
lower_6_8, upper_6_8 = outliers_iqr(total_6[total_6['month'] == 8]['공급량'])
lower_6_9, upper_6_9 = outliers_iqr(total_6[total_6['month'] == 9]['공급량'])
lower_6_10, upper_6_10 = outliers_iqr(total_6[total_6['month'] == 10]['공급량'])
lower_6_11, upper_6_11 = outliers_iqr(total_6[total_6['month'] == 11]['공급량'])
lower_6_12, upper_6_12 = outliers_iqr(total_6[total_6['month'] == 12]['공급량'])

*   total 테이블에 lower(Q1), upper(Q3) 빈 열 추가

In [ ]:
total['lower'] = ""
total['upper'] = ""

*   각 행별로 구분과 월에 맞는 lower, upper값 삽입

In [ ]:
total.loc[(total['구분'] == 0) & (total['month'] == 1), 'lower'] = lower_0_1
total.loc[(total['구분'] == 0) & (total['month'] == 2), 'lower'] = lower_0_2
total.loc[(total['구분'] == 0) & (total['month'] == 3), 'lower'] = lower_0_3
total.loc[(total['구분'] == 0) & (total['month'] == 4), 'lower'] = lower_0_4
total.loc[(total['구분'] == 0) & (total['month'] == 5), 'lower'] = lower_0_5
total.loc[(total['구분'] == 0) & (total['month'] == 6), 'lower'] = lower_0_6
total.loc[(total['구분'] == 0) & (total['month'] == 7), 'lower'] = lower_0_7
total.loc[(total['구분'] == 0) & (total['month'] == 8), 'lower'] = lower_0_8
total.loc[(total['구분'] == 0) & (total['month'] == 9), 'lower'] = lower_0_9
total.loc[(total['구분'] == 0) & (total['month'] == 10), 'lower'] = lower_0_10
total.loc[(total['구분'] == 0) & (total['month'] == 11), 'lower'] = lower_0_11
total.loc[(total['구분'] == 0) & (total['month'] == 12), 'lower'] = lower_0_12

total.loc[(total['구분'] == 0) & (total['month'] == 1), 'upper'] = upper_0_1
total.loc[(total['구분'] == 0) & (total['month'] == 2), 'upper'] = upper_0_2
total.loc[(total['구분'] == 0) & (total['month'] == 3), 'upper'] = upper_0_3
total.loc[(total['구분'] == 0) & (total['month'] == 4), 'upper'] = upper_0_4
total.loc[(total['구분'] == 0) & (total['month'] == 5), 'upper'] = upper_0_5
total.loc[(total['구분'] == 0) & (total['month'] == 6), 'upper'] = upper_0_6
total.loc[(total['구분'] == 0) & (total['month'] == 7), 'upper'] = upper_0_7
total.loc[(total['구분'] == 0) & (total['month'] == 8), 'upper'] = upper_0_8
total.loc[(total['구분'] == 0) & (total['month'] == 9), 'upper'] = upper_0_9
total.loc[(total['구분'] == 0) & (total['month'] == 10), 'upper'] = upper_0_10
total.loc[(total['구분'] == 0) & (total['month'] == 11), 'upper'] = upper_0_11
total.loc[(total['구분'] == 0) & (total['month'] == 12), 'upper'] = upper_0_12

In [ ]:
total.loc[(total['구분'] == 1) & (total['month'] == 1), 'lower'] = lower_0_1
total.loc[(total['구분'] == 1) & (total['month'] == 2), 'lower'] = lower_0_2
total.loc[(total['구분'] == 1) & (total['month'] == 3), 'lower'] = lower_0_3
total.loc[(total['구분'] == 1) & (total['month'] == 4), 'lower'] = lower_0_4
total.loc[(total['구분'] == 1) & (total['month'] == 5), 'lower'] = lower_0_5
total.loc[(total['구분'] == 1) & (total['month'] == 6), 'lower'] = lower_0_6
total.loc[(total['구분'] == 1) & (total['month'] == 7), 'lower'] = lower_0_7
total.loc[(total['구분'] == 1) & (total['month'] == 8), 'lower'] = lower_0_8
total.loc[(total['구분'] == 1) & (total['month'] == 9), 'lower'] = lower_0_9
total.loc[(total['구분'] == 1) & (total['month'] == 10), 'lower'] = lower_0_10
total.loc[(total['구분'] == 1) & (total['month'] == 11), 'lower'] = lower_0_11
total.loc[(total['구분'] == 1) & (total['month'] == 12), 'lower'] = lower_0_12

total.loc[(total['구분'] == 1) & (total['month'] == 1), 'upper'] = upper_1_1
total.loc[(total['구분'] == 1) & (total['month'] == 2), 'upper'] = upper_1_2
total.loc[(total['구분'] == 1) & (total['month'] == 3), 'upper'] = upper_1_3
total.loc[(total['구분'] == 1) & (total['month'] == 4), 'upper'] = upper_1_4
total.loc[(total['구분'] == 1) & (total['month'] == 5), 'upper'] = upper_1_5
total.loc[(total['구분'] == 1) & (total['month'] == 6), 'upper'] = upper_1_6
total.loc[(total['구분'] == 1) & (total['month'] == 7), 'upper'] = upper_1_7
total.loc[(total['구분'] == 1) & (total['month'] == 8), 'upper'] = upper_1_8
total.loc[(total['구분'] == 1) & (total['month'] == 9), 'upper'] = upper_1_9
total.loc[(total['구분'] == 1) & (total['month'] == 10), 'upper'] = upper_1_10
total.loc[(total['구분'] == 1) & (total['month'] == 11), 'upper'] = upper_1_11
total.loc[(total['구분'] == 1) & (total['month'] == 12), 'upper'] = upper_1_12

In [ ]:
total.loc[(total['구분'] == 2) & (total['month'] == 1), 'lower'] = lower_2_1
total.loc[(total['구분'] == 2) & (total['month'] == 2), 'lower'] = lower_2_2
total.loc[(total['구분'] == 2) & (total['month'] == 3), 'lower'] = lower_2_3
total.loc[(total['구분'] == 2) & (total['month'] == 4), 'lower'] = lower_2_4
total.loc[(total['구분'] == 2) & (total['month'] == 5), 'lower'] = lower_2_5
total.loc[(total['구분'] == 2) & (total['month'] == 6), 'lower'] = lower_2_6
total.loc[(total['구분'] == 2) & (total['month'] == 7), 'lower'] = lower_2_7
total.loc[(total['구분'] == 2) & (total['month'] == 8), 'lower'] = lower_2_8
total.loc[(total['구분'] == 2) & (total['month'] == 9), 'lower'] = lower_2_9
total.loc[(total['구분'] == 2) & (total['month'] == 10), 'lower'] = lower_2_10
total.loc[(total['구분'] == 2) & (total['month'] == 11), 'lower'] = lower_2_11
total.loc[(total['구분'] == 2) & (total['month'] == 12), 'lower'] = lower_2_12

total.loc[(total['구분'] == 2) & (total['month'] == 1), 'upper'] = upper_2_1
total.loc[(total['구분'] == 2) & (total['month'] == 2), 'upper'] = upper_2_2
total.loc[(total['구분'] == 2) & (total['month'] == 3), 'upper'] = upper_2_3
total.loc[(total['구분'] == 2) & (total['month'] == 4), 'upper'] = upper_2_4
total.loc[(total['구분'] == 2) & (total['month'] == 5), 'upper'] = upper_2_5
total.loc[(total['구분'] == 2) & (total['month'] == 6), 'upper'] = upper_2_6
total.loc[(total['구분'] == 2) & (total['month'] == 7), 'upper'] = upper_2_7
total.loc[(total['구분'] == 2) & (total['month'] == 8), 'upper'] = upper_2_8
total.loc[(total['구분'] == 2) & (total['month'] == 9), 'upper'] = upper_2_9
total.loc[(total['구분'] == 2) & (total['month'] == 10), 'upper'] = upper_2_10
total.loc[(total['구분'] == 2) & (total['month'] == 11), 'upper'] = upper_2_11
total.loc[(total['구분'] == 2) & (total['month'] == 12), 'upper'] = upper_2_12

In [ ]:
total.loc[(total['구분'] == 3) & (total['month'] == 1), 'lower'] = lower_3_1
total.loc[(total['구분'] == 3) & (total['month'] == 2), 'lower'] = lower_3_2
total.loc[(total['구분'] == 3) & (total['month'] == 3), 'lower'] = lower_3_3
total.loc[(total['구분'] == 3) & (total['month'] == 4), 'lower'] = lower_3_4
total.loc[(total['구분'] == 3) & (total['month'] == 5), 'lower'] = lower_3_5
total.loc[(total['구분'] == 3) & (total['month'] == 6), 'lower'] = lower_3_6
total.loc[(total['구분'] == 3) & (total['month'] == 7), 'lower'] = lower_3_7
total.loc[(total['구분'] == 3) & (total['month'] == 8), 'lower'] = lower_3_8
total.loc[(total['구분'] == 3) & (total['month'] == 9), 'lower'] = lower_3_9
total.loc[(total['구분'] == 3) & (total['month'] == 10), 'lower'] = lower_3_10
total.loc[(total['구분'] == 3) & (total['month'] == 11), 'lower'] = lower_3_11
total.loc[(total['구분'] == 3) & (total['month'] == 12), 'lower'] = lower_3_12

total.loc[(total['구분'] == 3) & (total['month'] == 1), 'upper'] = upper_3_1
total.loc[(total['구분'] == 3) & (total['month'] == 2), 'upper'] = upper_3_2
total.loc[(total['구분'] == 3) & (total['month'] == 3), 'upper'] = upper_3_3
total.loc[(total['구분'] == 3) & (total['month'] == 4), 'upper'] = upper_3_4
total.loc[(total['구분'] == 3) & (total['month'] == 5), 'upper'] = upper_3_5
total.loc[(total['구분'] == 3) & (total['month'] == 6), 'upper'] = upper_3_6
total.loc[(total['구분'] == 3) & (total['month'] == 7), 'upper'] = upper_3_7
total.loc[(total['구분'] == 3) & (total['month'] == 8), 'upper'] = upper_3_8
total.loc[(total['구분'] == 3) & (total['month'] == 9), 'upper'] = upper_3_9
total.loc[(total['구분'] == 3) & (total['month'] == 10), 'upper'] = upper_3_10
total.loc[(total['구분'] == 3) & (total['month'] == 11), 'upper'] = upper_3_11
total.loc[(total['구분'] == 3) & (total['month'] == 12), 'upper'] = upper_3_12

In [ ]:
total.loc[(total['구분'] == 4) & (total['month'] == 1), 'lower'] = lower_4_1
total.loc[(total['구분'] == 4) & (total['month'] == 2), 'lower'] = lower_4_2
total.loc[(total['구분'] == 4) & (total['month'] == 3), 'lower'] = lower_4_3
total.loc[(total['구분'] == 4) & (total['month'] == 4), 'lower'] = lower_4_4
total.loc[(total['구분'] == 4) & (total['month'] == 5), 'lower'] = lower_4_5
total.loc[(total['구분'] == 4) & (total['month'] == 6), 'lower'] = lower_4_6
total.loc[(total['구분'] == 4) & (total['month'] == 7), 'lower'] = lower_4_7
total.loc[(total['구분'] == 4) & (total['month'] == 8), 'lower'] = lower_4_8
total.loc[(total['구분'] == 4) & (total['month'] == 9), 'lower'] = lower_4_9
total.loc[(total['구분'] == 4) & (total['month'] == 10), 'lower'] = lower_4_10
total.loc[(total['구분'] == 4) & (total['month'] == 11), 'lower'] = lower_4_11
total.loc[(total['구분'] == 4) & (total['month'] == 12), 'lower'] = lower_4_12

total.loc[(total['구분'] == 4) & (total['month'] == 1), 'upper'] = upper_4_1
total.loc[(total['구분'] == 4) & (total['month'] == 2), 'upper'] = upper_4_2
total.loc[(total['구분'] == 4) & (total['month'] == 3), 'upper'] = upper_4_3
total.loc[(total['구분'] == 4) & (total['month'] == 4), 'upper'] = upper_4_4
total.loc[(total['구분'] == 4) & (total['month'] == 5), 'upper'] = upper_4_5
total.loc[(total['구분'] == 4) & (total['month'] == 6), 'upper'] = upper_4_6
total.loc[(total['구분'] == 4) & (total['month'] == 7), 'upper'] = upper_4_7
total.loc[(total['구분'] == 4) & (total['month'] == 8), 'upper'] = upper_4_8
total.loc[(total['구분'] == 4) & (total['month'] == 9), 'upper'] = upper_4_9
total.loc[(total['구분'] == 4) & (total['month'] == 10), 'upper'] = upper_4_10
total.loc[(total['구분'] == 4) & (total['month'] == 11), 'upper'] = upper_4_11
total.loc[(total['구분'] == 4) & (total['month'] == 12), 'upper'] = upper_4_12

In [ ]:
total.loc[(total['구분'] == 5) & (total['month'] == 1), 'lower'] = lower_5_1
total.loc[(total['구분'] == 5) & (total['month'] == 2), 'lower'] = lower_5_2
total.loc[(total['구분'] == 5) & (total['month'] == 3), 'lower'] = lower_5_3
total.loc[(total['구분'] == 5) & (total['month'] == 4), 'lower'] = lower_5_4
total.loc[(total['구분'] == 5) & (total['month'] == 5), 'lower'] = lower_5_5
total.loc[(total['구분'] == 5) & (total['month'] == 6), 'lower'] = lower_5_6
total.loc[(total['구분'] == 5) & (total['month'] == 7), 'lower'] = lower_5_7
total.loc[(total['구분'] == 5) & (total['month'] == 8), 'lower'] = lower_5_8
total.loc[(total['구분'] == 5) & (total['month'] == 9), 'lower'] = lower_5_9
total.loc[(total['구분'] == 5) & (total['month'] == 10), 'lower'] = lower_5_10
total.loc[(total['구분'] == 5) & (total['month'] == 11), 'lower'] = lower_5_11
total.loc[(total['구분'] == 5) & (total['month'] == 12), 'lower'] = lower_5_12

total.loc[(total['구분'] == 5) & (total['month'] == 1), 'upper'] = upper_5_1
total.loc[(total['구분'] == 5) & (total['month'] == 2), 'upper'] = upper_5_2
total.loc[(total['구분'] == 5) & (total['month'] == 3), 'upper'] = upper_5_3
total.loc[(total['구분'] == 5) & (total['month'] == 4), 'upper'] = upper_5_4
total.loc[(total['구분'] == 5) & (total['month'] == 5), 'upper'] = upper_5_5
total.loc[(total['구분'] == 5) & (total['month'] == 6), 'upper'] = upper_5_6
total.loc[(total['구분'] == 5) & (total['month'] == 7), 'upper'] = upper_5_7
total.loc[(total['구분'] == 5) & (total['month'] == 8), 'upper'] = upper_5_8
total.loc[(total['구분'] == 5) & (total['month'] == 9), 'upper'] = upper_5_9
total.loc[(total['구분'] == 5) & (total['month'] == 10), 'upper'] = upper_5_10
total.loc[(total['구분'] == 5) & (total['month'] == 11), 'upper'] = upper_5_11
total.loc[(total['구분'] == 5) & (total['month'] == 12), 'upper'] = upper_5_12

In [ ]:
total.loc[(total['구분'] == 6) & (total['month'] == 1), 'lower'] = lower_6_1
total.loc[(total['구분'] == 6) & (total['month'] == 2), 'lower'] = lower_6_2
total.loc[(total['구분'] == 6) & (total['month'] == 3), 'lower'] = lower_6_3
total.loc[(total['구분'] == 6) & (total['month'] == 4), 'lower'] = lower_6_4
total.loc[(total['구분'] == 6) & (total['month'] == 5), 'lower'] = lower_6_5
total.loc[(total['구분'] == 6) & (total['month'] == 6), 'lower'] = lower_6_6
total.loc[(total['구분'] == 6) & (total['month'] == 7), 'lower'] = lower_6_7
total.loc[(total['구분'] == 6) & (total['month'] == 8), 'lower'] = lower_6_8
total.loc[(total['구분'] == 6) & (total['month'] == 9), 'lower'] = lower_6_9
total.loc[(total['구분'] == 6) & (total['month'] == 10), 'lower'] = lower_6_10
total.loc[(total['구분'] == 6) & (total['month'] == 11), 'lower'] = lower_6_11
total.loc[(total['구분'] == 6) & (total['month'] == 12), 'lower'] = lower_6_12

total.loc[(total['구분'] == 6) & (total['month'] == 1), 'upper'] = upper_6_1
total.loc[(total['구분'] == 6) & (total['month'] == 2), 'upper'] = upper_6_2
total.loc[(total['구분'] == 6) & (total['month'] == 3), 'upper'] = upper_6_3
total.loc[(total['구분'] == 6) & (total['month'] == 4), 'upper'] = upper_6_4
total.loc[(total['구분'] == 6) & (total['month'] == 5), 'upper'] = upper_6_5
total.loc[(total['구분'] == 6) & (total['month'] == 6), 'upper'] = upper_6_6
total.loc[(total['구분'] == 6) & (total['month'] == 7), 'upper'] = upper_6_7
total.loc[(total['구분'] == 6) & (total['month'] == 8), 'upper'] = upper_6_8
total.loc[(total['구분'] == 6) & (total['month'] == 9), 'upper'] = upper_6_9
total.loc[(total['구분'] == 6) & (total['month'] == 10), 'upper'] = upper_6_10
total.loc[(total['구분'] == 6) & (total['month'] == 11), 'upper'] = upper_6_11
total.loc[(total['구분'] == 6) & (total['month'] == 12), 'upper'] = upper_6_12

*   total에 이상치 공급량 열 추가
    - 공급량이 구분에 따른 월별 Q1보다 낮으면 Q1값으로 대체
    - 공급량이 구분에 따른 월별 Q3보다 높으면 Q3값으로 대체

In [ ]:
total['이상치처리_공급량'] = ""

In [ ]:
total['이상치처리_공급량'] = total.apply(lambda x: x['lower'] if x['공급량'] < x['lower'] else x['upper'] if x['공급량'] > x['upper'] else x['공급량'], axis=1)

In [ ]:
total = total.drop(['lower', 'upper'], axis = 1)
total

,연월일,시간,구분,공급량,year,month,day,weekday,이상치처리_공급량
0,2013-01-01,1,0,2497.129,2013,1,1,1,2497.129
1,2013-01-01,2,0,2363.265,2013,1,1,1,2363.265
2,2013-01-01,3,0,2258.505,2013,1,1,1,2258.505
3,2013-01-01,4,0,2243.969,2013,1,1,1,2243.969
4,2013-01-01,5,0,2344.105,2013,1,1,1,2344.105
...,...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31,0,681.033
368084,2018-12-31,21,6,669.961,2018,12,31,0,669.961
368085,2018-12-31,22,6,657.941,2018,12,31,0,657.941
368086,2018-12-31,23,6,610.953,2018,12,31,0,610.953


## Feature 추가

**공급량 평균**
*   공급량_평균: 구분, month, day, 시간별 전체 기간 동안의 공급량 평균
*   요일별_평균: 구분, month, 요일별, 시간별 전체 기간 동안의 공급량 평균

In [ ]:
## 공급량_평균: 구분, month, day, 시간별 전체 기간 동안의 공급량 평균
supply_mean = pd.pivot_table(total, values = '공급량', index=['구분', 'month', 'day', '시간'], aggfunc = np.mean).reset_index()
tqdm.pandas()
total['공급량_평균'] = total.progress_apply(lambda x: supply_mean.loc[(supply_mean.구분 == x['구분'])&(supply_mean.month == x['month'])&(supply_mean.day == x['day'])&(supply_mean.시간 == x['시간']), '공급량'].values[0], axis = 1)

## 요일별_평균: 구분, month, 요일별, 시간별 전체 기간 동안의 공급량 평균
supply_weekday_mean = pd.pivot_table(total, values = '공급량', index=['구분', 'month', 'weekday', '시간'], aggfunc = np.mean).reset_index()
tqdm.pandas()
total['공급량_요일_평균'] = total.progress_apply(lambda x: supply_weekday_mean.loc[(supply_weekday_mean.구분 == x['구분'])&(supply_weekday_mean.month == x['month'])&(supply_weekday_mean.weekday == x['weekday'])&(supply_weekday_mean.시간 == x['시간']), '공급량'].values[0], axis = 1)

100%|██████████| 368088/368088 [11:57<00:00, 512.69it/s]


**전년도 공급량**
- total 테이블을 구분, month, day, 시간에 따라 정렬한 테이블 total2 생성
- 같은 구분, 월, 일, 시간의 데이터들이 년도별로 정렬
- shift(1)로 하나씩 내리면 같은 구분, 월, 일, 시간의 전년도 공급량을 가질 수 있음
- 이때 2013년도 행에는 2018년도 데이터가 들어가기 때문에 2013년의 전년도 공급량은 해당 년도의 공급량으로 대체

In [ ]:
total2 = total.sort_values(by=['구분','month','day','시간'])

In [ ]:
total2['전년도 공급량'] = total2['공급량'].shift(1)

In [ ]:
total['전년도_공급량'] = total.progress_apply(lambda x: total2.loc[(total2.구분 == x['구분'])&(total2.연월일 == x['연월일'])&(total2.시간 == x['시간']), '전년도 공급량'].values[0], axis = 1)

100%|██████████| 368088/368088 [19:36<00:00, 312.80it/s]


In [ ]:
total['전년도_공급량'] = total.apply(lambda x: x['공급량'] if x['year'] == 2013 else x['전년도_공급량'], axis=1)

**sin_time, cos_time**

In [ ]:
total['sin_time'] = np.sin(2*np.pi*total.시간/24)
total['cos_time'] = np.cos(2*np.pi*total.시간/24)

**더미변수**

In [ ]:
# 겨울 더미변수
total['계절'] = total['month'].apply(lambda month_number:(month_number%12 +3)//3)
winter = [1]
total['겨울_더미'] = total['계절'].apply(lambda x : 1 if x in winter else 0)
total = total.drop(['계절'], axis = 1)

# 특정 시간 더미변수
hour_list = [1, 2, 3, 4, 5, 6, 7, 16, 17, 22, 23, 24]
total['특정시간_더미'] = total['시간'].apply(lambda x : 0 if x in hour_list else 1)

# 평일 더미변수
weekend = [5,6]
total['평일'] = total['weekday'].apply(lambda x : 0 if x in weekend else 1)

# G 공급사 더미변수
the_most = [5]
total['G_공급사'] = total['구분'].apply(lambda x : 1 if x in the_most else 0)

**외부데이터**

In [ ]:
# 가스기름상대가격
tqdm.pandas()
total['가스기름상대가격'] = total.progress_apply(lambda x: GO_price.loc[(GO_price.year == x['year'])&(GO_price.month == x['month']), '상대가격'].values[0], axis = 1)

# 가스전기상대가격
tqdm.pandas()
total['가스전기상대가격'] = total.progress_apply(lambda x: GE_price.loc[(GE_price.year == x['year'])&(GE_price.month == x['month']), '상대가격'].values[0], axis = 1)

# 전산업생산지수
tqdm.pandas()
total['전산업생산지수'] = total.progress_apply(lambda x: IAIP.loc[(IAIP.year == x['year'])&(IAIP.month == x['month']), '전산업생산지수(농립어업 제외)'].values[0], axis = 1)

# 가스수요산업지수
tqdm.pandas()
total['가스수요산업지수'] = total.progress_apply(lambda x: gas_d.loc[(gas_d.연 == x['year'])&(gas_d.월 == x['month']), '가스수요산업지수'].values[0], axis = 1)

# 휴일, 주말, 휴일_주말
## 2013~2018년 공휴일
holidays = pd.concat([pd.Series(np.array(SouthKorea().holidays(2018))[:, 0]), pd.Series(np.array(SouthKorea().holidays(2017))[:, 0]), pd.Series(np.array(SouthKorea().holidays(2016))[:, 0]), pd.Series(np.array(SouthKorea().holidays(2015))[:, 0]), pd.Series(np.array(SouthKorea().holidays(2014))[:, 0]), pd.Series(np.array(SouthKorea().holidays(2013))[:, 0])]).reset_index(drop=True)
total['휴일'] = total['연월일'].dt.date.isin(holidays).astype(int)
total['주말'] = total['weekday'].map({0:0, 1:0, 2:0, 3:0, 4:0, 5:1, 6:1})
total['휴일_주말'] = (total['주말'] + total['휴일']).map({0:0, 1:1, 2:1})

100%|██████████| 368088/368088 [04:17<00:00, 1430.50it/s]


**기온변수**

In [ ]:
temp_mean2013 = pd.pivot_table(weather_2013, values = '기온(°C)', index=['일시'], aggfunc = np.mean).reset_index()
temp_mean2014 = pd.pivot_table(weather_2014, values = '기온(°C)', index=['일시'], aggfunc = np.mean).reset_index()
temp_mean2015 = pd.pivot_table(weather_2015, values = '기온(°C)', index=['일시'], aggfunc = np.mean).reset_index()
temp_mean2016 = pd.pivot_table(weather_2016, values = '기온(°C)', index=['일시'], aggfunc = np.mean).reset_index()
temp_mean2017 = pd.pivot_table(weather_2017, values = '기온(°C)', index=['일시'], aggfunc = np.mean).reset_index()
temp_mean2018 = pd.pivot_table(weather_2018, values = '기온(°C)', index=['일시'], aggfunc = np.mean).reset_index()

In [ ]:
temp = pd.DataFrame({'일시':['2015-01-01 00:00'], '기온(°C)':[-3.340556]})
temp_mean2015 = pd.concat([temp, temp_mean2015])

In [ ]:
def df_duplicate(df):
  new_df = pd.DataFrame()
  for _ in range(7):
    new_df = pd.concat([new_df, df])
  return new_df

In [ ]:
weather_2013_total = df_duplicate(temp_mean2013)
weather_2014_total = df_duplicate(temp_mean2014)
weather_2015_total = df_duplicate(temp_mean2015)
weather_2016_total = df_duplicate(temp_mean2016)
weather_2017_total = df_duplicate(temp_mean2017)
weather_2018_total = df_duplicate(temp_mean2018)

In [ ]:
weather_total = pd.concat([weather_2013_total, weather_2014_total, weather_2015_total, weather_2016_total, weather_2017_total, weather_2018_total])

In [ ]:
total

,연월일,시간,구분,공급량,year,month,day,weekday,이상치처리_공급량,공급량_평균,공급량_요일_평균,전년도_공급량,sin_time,cos_time,겨울_더미,특정시간_더미,평일,G_공급사,가스기름상대가격,가스전기상대가격,전산업생산지수,가스수요산업지수,휴일,주말,휴일_주말
0,2013-01-01,1,0,2497.129,2013,1,1,1,2497.129,1913.798167,2091.462444,2497.129,2.588190e-01,0.965926,1,0,1,0,0.66,108.59,94.3,122.03,1,0,1
1,2013-01-01,2,0,2363.265,2013,1,1,1,2363.265,1801.091833,1845.728111,2363.265,5.000000e-01,0.866025,1,0,1,0,0.66,108.59,94.3,122.03,1,0,1
2,2013-01-01,3,0,2258.505,2013,1,1,1,2258.505,1696.859000,1729.965741,2258.505,7.071068e-01,0.707107,1,0,1,0,0.66,108.59,94.3,122.03,1,0,1
3,2013-01-01,4,0,2243.969,2013,1,1,1,2243.969,1689.182667,1783.386852,2243.969,8.660254e-01,0.500000,1,0,1,0,0.66,108.59,94.3,122.03,1,0,1
4,2013-01-01,5,0,2344.105,2013,1,1,1,2344.105,1796.721167,1975.943778,2344.105,9.659258e-01,0.258819,1,0,1,0,0.66,108.59,94.3,122.03,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31,0,681.033,575.836167,577.313222,517.264,-8.660254e-01,0.500000,1,1,1,0,0.74,92.69,116.7,95.56,0,0,0
368084,2018-12-31,21,6,669.961,2018,12,31,0,669.961,567.201500,583.881037,530.896,-7.071068e-01,0.707107,1,1,1,0,0.74,92.69,116.7,95.56,0,0,0
368085,2018-12-31,22,6,657.941,2018,12,31,0,657.941,549.175333,567.648000,506.287,-5.000000e-01,0.866025,1,0,1,0,0.74,92.69,116.7,95.56,0,0,0
368086,2018-12-31,23,6,610.953,2018,12,31,0,610.953,512.990500,522.023185,470.638,-2.588190e-01,0.965926,1,0,1,0,0.74,92.69,116.7,95.56,0,0,0


In [ ]:
weather_total

,일시,기온(°C)
0,2013-01-01 00:00,-7.055682
1,2013-01-01 01:00,-7.464773
2,2013-01-01 02:00,-7.705682
3,2013-01-01 03:00,-7.813636
4,2013-01-01 04:00,-8.031818
...,...,...
8755,2018-12-31 19:00,-0.964835
8756,2018-12-31 20:00,-1.726374
8757,2018-12-31 21:00,-2.404396
8758,2018-12-31 22:00,-3.031868


In [ ]:
total['월평균_기온'] = weather_total['기온(°C)'].values

In [ ]:
total

,연월일,시간,구분,공급량,year,month,day,weekday,이상치처리_공급량,공급량_평균,공급량_요일_평균,전년도_공급량,sin_time,cos_time,겨울_더미,특정시간_더미,평일,G_공급사,가스기름상대가격,가스전기상대가격,전산업생산지수,가스수요산업지수,휴일,주말,휴일_주말,월평균_기온
0,2013-01-01,1,0,2497.129,2013,1,1,1,2497.129,1913.798167,2091.462444,2497.129,2.588190e-01,0.965926,1,0,1,0,0.66,108.59,94.3,122.03,1,0,1,-7.055682
1,2013-01-01,2,0,2363.265,2013,1,1,1,2363.265,1801.091833,1845.728111,2363.265,5.000000e-01,0.866025,1,0,1,0,0.66,108.59,94.3,122.03,1,0,1,-7.464773
2,2013-01-01,3,0,2258.505,2013,1,1,1,2258.505,1696.859000,1729.965741,2258.505,7.071068e-01,0.707107,1,0,1,0,0.66,108.59,94.3,122.03,1,0,1,-7.705682
3,2013-01-01,4,0,2243.969,2013,1,1,1,2243.969,1689.182667,1783.386852,2243.969,8.660254e-01,0.500000,1,0,1,0,0.66,108.59,94.3,122.03,1,0,1,-7.813636
4,2013-01-01,5,0,2344.105,2013,1,1,1,2344.105,1796.721167,1975.943778,2344.105,9.659258e-01,0.258819,1,0,1,0,0.66,108.59,94.3,122.03,1,0,1,-8.031818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31,0,681.033,575.836167,577.313222,517.264,-8.660254e-01,0.500000,1,1,1,0,0.74,92.69,116.7,95.56,0,0,0,-0.964835
368084,2018-12-31,21,6,669.961,2018,12,31,0,669.961,567.201500,583.881037,530.896,-7.071068e-01,0.707107,1,1,1,0,0.74,92.69,116.7,95.56,0,0,0,-1.726374
368085,2018-12-31,22,6,657.941,2018,12,31,0,657.941,549.175333,567.648000,506.287,-5.000000e-01,0.866025,1,0,1,0,0.74,92.69,116.7,95.56,0,0,0,-2.404396
368086,2018-12-31,23,6,610.953,2018,12,31,0,610.953,512.990500,522.023185,470.638,-2.588190e-01,0.965926,1,0,1,0,0.74,92.69,116.7,95.56,0,0,0,-3.031868


# 3. Test set 전처리

In [ ]:
test = pd.read_csv(path+'/test.csv')
submission = pd.read_csv(path+'/sample_submission.csv')

**전처리**

In [ ]:
test['일자'] = test['일자|시간|구분'].str.split(' ').str[0]
test['시간'] = test['일자|시간|구분'].str.split(' ').str[1].astype(int)
test['구분'] = test['일자|시간|구분'].str.split(' ').str[2]

In [ ]:
test['일자'] = pd.to_datetime(test['일자'])
test['year'] = test['일자'].dt.year
test['month'] = test['일자'].dt.month
test['day'] = test['일자'].dt.day
test['weekday'] = test['일자'].dt.weekday

In [ ]:
d_map = {}
for i, d in enumerate(test['구분'].unique()):
    d_map[d] = i
test['구분'] = test['구분'].map(d_map)

**Feature 추가**

In [ ]:
# 공급량_평균
test['공급량_평균'] = test.progress_apply(lambda x: supply_mean.loc[(supply_mean.구분 == x['구분'])&(supply_mean.month == x['month'])&(supply_mean.day == x['day'])&(supply_mean.시간 == x['시간']), '공급량'].values[0], axis = 1)

# 공급량_요일_평균
test['공급량_요일_평균'] = test.progress_apply(lambda x: supply_weekday_mean.loc[(supply_weekday_mean.구분 == x['구분'])&(supply_weekday_mean.month == x['month'])&(supply_weekday_mean.weekday == x['weekday'])
&(supply_weekday_mean.시간 == x['시간']), '공급량'].values[0], axis = 1)

# 전년도_공급량
total_2018 = total[total.year == 2018]
test['전년도_공급량'] = test.progress_apply(lambda x: total_2018.loc[(total_2018.구분 == x['구분'])&(total_2018.month == x['month'])&(total_2018.day == x['day'])&(total_2018.시간 == x['시간']), '공급량'].values[0], axis = 1)

# sin_time, cos_time
test['sin_time'] = np.sin(2*np.pi*total.시간/24)
test['cos_time'] = np.cos(2*np.pi*total.시간/24)

# 겨울_더미
test['계절'] = test['month'].apply(lambda month_number:(month_number%12 +3)//3)
winter = [1]
test['겨울_더미'] = test['계절'].apply(lambda x : 1 if x in winter else 0)
test = test.drop(['계절'], axis = 1)

# 특정시간_더미
test['특정시간_더미'] = test['시간'].apply(lambda x : 0 if x in hour_list else 1)

# 평일
weekend = [5,6]
test['평일'] = test['weekday'].apply(lambda x : 0 if x in weekend else 1)

# G_공급사
the_most = [5]
test['G_공급사'] = test['구분'].apply(lambda x : 1 if x in the_most else 0)

# 가스기름상대가격
test['가스기름상대가격'] = test.progress_apply(lambda x: GO_price.loc[(GO_price.year == 2018)&(GO_price.month == 12), '상대가격'].values[0], axis = 1)

# 가스전기상대가격
test['가스전기상대가격'] = test.progress_apply(lambda x: GE_price.loc[(GE_price.year == 2018)&(GE_price.month == 12), '상대가격'].values[0], axis = 1)

# 전산업생산지수
test['전산업생산지수'] = test.progress_apply(lambda x: IAIP.loc[(IAIP.year == 2018)&(IAIP.month == 12), '전산업생산지수(농립어업 제외)'].values[0], axis = 1)

# 가스수요산업지수
total_2018_1 = total[(total.year == 2018) & (total.month ==1)]
total_2018_2 = total[(total.year == 2018) & (total.month ==2)]
total_2018_3 = total[(total.year == 2018) & (total.month ==3)]
total_1to3 = pd.concat([total_2018_1, total_2018_2,total_2018_3])
test['가스수요산업지수']= total_1to3['가스수요산업지수'].values

# 휴일
## 2019년 공휴일
holidays_2019 = pd.concat([pd.Series(np.array(SouthKorea().holidays(2019))[:, 0])]).reset_index(drop=True)
test['휴일'] = test['일자'].dt.date.isin(holidays_2019).astype(int)

# 주말
test['주말'] = test['weekday'].map({0:0, 1:0, 2:0, 3:0, 4:0, 5:1, 6:1})

# 휴일_주말
test['휴일_주말'] = (test['주말'] + test['휴일']).map({0:0, 1:1, 2:1})

100%|██████████| 15120/15120 [00:09<00:00, 1516.49it/s]


2018년 12월 21일에 배포된 2019년 1월~3월 월평균 기온 전망 정보를 이용해 test 데이터의 월평균 기온 변수 생성  

In [ ]:
# 월평균_기온
test['월평균_기온'] = ""
test.loc[(test['month'] == 1), ['월평균_기온']] = -1.0
test.loc[(test['month'] == 2), ['월평균_기온']] = 1.1
test.loc[(test['month'] == 3), ['월평균_기온']] = 6.0

In [ ]:
test.월평균_기온 = test.월평균_기온.astype(float)

In [ ]:
test

,일자|시간|구분,일자,시간,구분,year,month,day,weekday,공급량_평균,공급량_요일_평균,전년도_공급량,sin_time,cos_time,겨울_더미,특정시간_더미,평일,G_공급사,가스기름상대가격,가스전기상대가격,전산업생산지수,가스수요산업지수,휴일,주말,휴일_주말,월평균_기온
0,2019-01-01 01 A,2019-01-01,1,0,2019,1,1,1,1913.798167,2091.462444,1765.008,2.588190e-01,0.965926,1,0,1,0,0.74,92.69,116.7,94.88,1,0,1,-1.0
1,2019-01-01 02 A,2019-01-01,2,0,2019,1,1,1,1801.091833,1845.728111,1679.186,5.000000e-01,0.866025,1,0,1,0,0.74,92.69,116.7,94.88,1,0,1,-1.0
2,2019-01-01 03 A,2019-01-01,3,0,2019,1,1,1,1696.859000,1729.965741,1610.885,7.071068e-01,0.707107,1,0,1,0,0.74,92.69,116.7,94.88,1,0,1,-1.0
3,2019-01-01 04 A,2019-01-01,4,0,2019,1,1,1,1689.182667,1783.386852,1604.123,8.660254e-01,0.500000,1,0,1,0,0.74,92.69,116.7,94.88,1,0,1,-1.0
4,2019-01-01 05 A,2019-01-01,5,0,2019,1,1,1,1796.721167,1975.943778,1711.506,9.659258e-01,0.258819,1,0,1,0,0.74,92.69,116.7,94.88,1,0,1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15115,2019-03-31 20 H,2019-03-31,20,6,2019,3,31,6,316.634333,423.036667,244.162,-8.660254e-01,0.500000,0,1,0,0,0.74,92.69,116.7,104.06,0,1,1,6.0
15116,2019-03-31 21 H,2019-03-31,21,6,2019,3,31,6,327.582500,428.752407,248.059,-7.071068e-01,0.707107,0,1,0,0,0.74,92.69,116.7,104.06,0,1,1,6.0
15117,2019-03-31 22 H,2019-03-31,22,6,2019,3,31,6,311.249500,394.682074,231.181,-5.000000e-01,0.866025,0,0,0,0,0.74,92.69,116.7,104.06,0,1,1,6.0
15118,2019-03-31 23 H,2019-03-31,23,6,2019,3,31,6,277.961667,352.585778,199.022,-2.588190e-01,0.965926,0,0,0,0,0.74,92.69,116.7,104.06,0,1,1,6.0


# 4. 모델링

**데이터 세팅**

In [ ]:
# train/val set 분리
train_years = [2013,2014,2015,2016,2017]
val_years = [2018]

In [ ]:
train = total[total['year'].isin(train_years)]
val = total[total['year'].isin(val_years)]

In [ ]:
features = ['시간', '구분', 'month', 'day', 'weekday', 'sin_time', 'cos_time', '공급량_평균', '공급량_요일_평균', '전년도_공급량','가스기름상대가격', '가스전기상대가격', 
            '전산업생산지수','겨울_더미', '특정시간_더미', '평일', 'G_공급사', '가스수요산업지수', '휴일', '주말', '휴일_주말', '월평균_기온']
train_x = train[features]
train_y = train['공급량']

val_x = val[features]
val_y = val['공급량']

## 모델1: XGBoost: 전체 학습 및 예측 & 구분별 학습 후 구분별 예측


### 1) 하이퍼파라미터 튜닝

**0단계: 파라미터 고정**

In [ ]:
dtrain = xgb.DMatrix(data=train_x, label=train_y)
dval_x = xgb.DMatrix(val_x)

In [ ]:
param = {
	'eta': 0.05,
  'num_boost_rounds': 3000,
	'eval_metric': 'mae',
	'seed': 1,
  'max_depth': 5,
	'min_child_weight': 1,
  'gamma': 0,
  'sub_sample': 0.8,
  'colsample_bytree': 0.8,
}

model = xgb.train(param, dtrain)

In [ ]:
preds = model.predict(dval_x)
score = mean_absolute_error(val_y, preds)
score

**1단계: max_depth & min_child_weight 튜닝**

In [ ]:
def objectiveXGB(trial, train_x = train_x, train_y = train_y, val_x = val_x, val_y = val_y):
		param = {
			'eta': 0.05,
  		'num_boost_rounds': 3000,
			'eval_metric': 'mae',
			'seed': 1,
			'max_depth': trial.suggest_int('max_depth', 3 ,18),
			'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
			'gamma': 0,
			'sub_sample': 0.8,
			'colsample_bytree': 0.8
		}

		dtrain = xgb.DMatrix(data=train_x, label=train_y)
		dval_x = xgb.DMatrix(val_x)

		model = xgb.train(param, dtrain)

		preds = model.predict(dval_x)
		
		score = mean_absolute_error(val_y, preds)
 
		return score

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objectiveXGB, n_trials = 50)
print('Best trial: score {},\nparams {}'.format(study.best_trial.value, study.best_trial.params))

In [ ]:
# 하이퍼파라미터별 중요도를 확인할 수 있는 그래프
optuna.visualization.plot_param_importances(study)

*   두 파라미터 중요도 차이가 너무 많이 나서 min_child_weight 다시 튜닝

In [ ]:
def objectiveXGB(trial, train_x = train_x, train_y = train_y, val_x = val_x, val_y = val_y):
		param = {
			'eta': 0.05,
  		'num_boost_rounds': 3000,
			'eval_metric': 'mae',
			'seed': 1,
			'max_depth': 10,
			'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
			'gamma': 0,
			'sub_sample': 0.8,
			'colsample_bytree': 0.8
		}

		dtrain = xgb.DMatrix(data=train_x, label=train_y)
		dval_x = xgb.DMatrix(val_x)

		model = xgb.train(param, dtrain)

		preds = model.predict(dval_x)
		
		score = mean_absolute_error(val_y, preds)
 
		return score

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objectiveXGB, n_trials = 50)
print('Best trial: score {},\nparams {}'.format(study.best_trial.value, study.best_trial.params))

**2단계: gamma 튜닝**

In [ ]:
def objectiveXGB(trial, train_x = train_x, train_y = train_y, val_x = val_x, val_y = val_y):
		param = {
			'eta': 0.05,
  		'num_boost_rounds': 3000,
			'eval_metric': 'mae',
			'seed': 1,
			'max_depth': 10,
			'min_child_weight': 164,
			'gamma': trial.suggest_categorical('gamma', [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]),
			'sub_sample': 0.8,
			'colsample_bytree': 0.8
		}

		dtrain = xgb.DMatrix(data=train_x, label=train_y)
		dval_x = xgb.DMatrix(val_x)

		model = xgb.train(param, dtrain)

		preds = model.predict(dval_x)
		
		score = mean_absolute_error(val_y, preds)
 
		return score

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objectiveXGB, n_trials = 50)
print('Best trial: score {},\nparams {}'.format(study.best_trial.value, study.best_trial.params))

**3단계: sub_sample & colsample_bytree**

In [ ]:
def objectiveXGB(trial, train_x = train_x, train_y = train_y, val_x = val_x, val_y = val_y):
		param = {
			'eta': 0.05,
			'num_boost_rounds': 3000,
			'eval_metric': 'mae',
			'seed': 1,
			'max_depth': 10,
			'min_child_weight': 164,
			'gamma': 0.4,
			'sub_sample': trial.suggest_categorical('sub_sample', [0.6, 0.65, 0.7, 0.75, 0.8, 0.85]),
			'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.6, 0.65, 0.7, 0.75, 0.8, 0.85])
		}

		dtrain = xgb.DMatrix(data=train_x, label=train_y)
		dval_x = xgb.DMatrix(val_x)

		model = xgb.train(param, dtrain)

		preds = model.predict(dval_x)
		
		score = mean_absolute_error(val_y, preds)
 
		return score

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objectiveXGB, n_trials = 50)
print('Best trial: score {},\nparams {}'.format(study.best_trial.value, study.best_trial.params))

In [ ]:
# 하이퍼파라미터별 중요도를 확인할 수 있는 그래프
optuna.visualization.plot_param_importances(study)

**4단계: subsample 추가 튜닝**

In [ ]:
def objectiveXGB(trial, train_x = train_x, train_y = train_y, val_x = val_x, val_y = val_y):
		param = {
			'eta': 0.05,
			'num_boost_rounds': 3000,
			'eval_metric': 'mae',
			'seed': 1,
			'max_depth': 10,
			'min_child_weight': 164,
			'gamma': 0.4,
			'sub_sample': trial.suggest_categorical('sub_sample', [0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8]),
			'colsample_bytree': 0.6
		}

		dtrain = xgb.DMatrix(data=train_x, label=train_y)
		dval_x = xgb.DMatrix(val_x)

		model = xgb.train(param, dtrain)

		preds = model.predict(dval_x)
		
		score = mean_absolute_error(val_y, preds)
 
		return score

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objectiveXGB, n_trials = 50)
print('Best trial: score {},\nparams {}'.format(study.best_trial.value, study.best_trial.params))

**5단계: alpha & lambda 튜닝**

In [ ]:
def objectiveXGB(trial, train_x = train_x, train_y = train_y, val_x = val_x, val_y = val_y):
		param = {
			'eta': 0.05,
			'num_boost_rounds': 3000,
			'eval_metric': 'mae',
			'seed': 1,
			'max_depth': 10,
			'min_child_weight': 164,
			'gamma': 0.4,
			'sub_sample': 0.4,
			'colsample_bytree': 0.6,
      'alpha': trial.suggest_categorical('alpha', [1e-5, 1e-2, 0.1, 1, 10, 20, 30, 50, 60, 70, 80, 90, 100]),
      'lamnbda': trial.suggest_categorical('lambda', [1e-5, 1e-2, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 0.6])                                                
		}


		dtrain = xgb.DMatrix(data=train_x, label=train_y)
		dval_x = xgb.DMatrix(val_x)

		model = xgb.train(param, dtrain)

		preds = model.predict(dval_x)
		
		score = mean_absolute_error(val_y, preds)
 
		return score

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objectiveXGB, n_trials = 50)
print('Best trial: score {},\nparams {}'.format(study.best_trial.value, study.best_trial.params))

### 2) 학습

In [ ]:
dtrain = xgb.DMatrix(data=train_x, label = train_y)
dval = xgb.DMatrix(data=val_x, label = val_y)
wlist = [(dtrain, 'train'), (dval,'eval')]
params = {
		'eta': 0.05,
		'num_boost_rounds': 3000,
		'eval_metric': 'mae',
		'seed': 1,
		'max_depth': 10,
		'min_child_weight': 164,
		'gamma': 0.4,
		'sub_sample': 0.4,
		'colsample_bytree': 0.6,
    'alpha': 20, 
    'lamnbda': 0.6                                                
}

model = xgb.train( params, dtrain, 500, evals=wlist, verbose_eval=20, early_stopping_rounds=100)

[0]	train-mae:887.758	eval-mae:964.479
Multiple eval metrics have been passed: 'eval-mae' will be used for early stopping.

Will train until eval-mae hasn't improved in 100 rounds.
[20]	train-mae:319.536	eval-mae:372.964
[40]	train-mae:120.571	eval-mae:164.479
[60]	train-mae:59.792	eval-mae:110.409
[80]	train-mae:44.1168	eval-mae:96.6311
[100]	train-mae:39.1217	eval-mae:92.2757
[120]	train-mae:36.1502	eval-mae:91.2457
[140]	train-mae:34.1316	eval-mae:90.9239
[160]	train-mae:32.8204	eval-mae:90.7355
[180]	train-mae:31.8133	eval-mae:90.7883
[200]	train-mae:31.1221	eval-mae:90.7646
[220]	train-mae:30.6272	eval-mae:90.8509
[240]	train-mae:30.3737	eval-mae:90.8479
[260]	train-mae:30.0169	eval-mae:90.8416
[280]	train-mae:29.6847	eval-mae:90.8288
Stopping. Best iteration:
[188]	train-mae:31.5166	eval-mae:90.6477



In [ ]:
test_x = test[features]
test_X = xgb.DMatrix(test_x)

pred = model.predict(test_X)

In [ ]:
submission_XGB_1 = submission
submission_XGB_1['공급량'] = pred

### 3) 구분별 학습



*   '구분'별로 이상치를 처리한 이상치처리_공급량을 y값으로 설정하여 구분별로 나눠서 학습



In [ ]:
pred = []
for i in tqdm(range(0,7)):
  train_x = train[train['구분']== i][features]
  train_y = train[train['구분']== i]['이상치처리_공급량']

  val_x = val[val['구분']== i][features]
  val_y = val[val['구분']== i]['이상치처리_공급량']

  dtrain = xgb.DMatrix(data=train_x, label = train_y)
  dval = xgb.DMatrix(data=val_x, label = val_y)
  wlist = [(dtrain, 'train'), (dval,'eval')]

  params = {
  		'eta': 0.05,
	  	'num_boost_rounds': 3000,
	  	'eval_metric': 'mae',
	  	'seed': 1,
	  	'max_depth': 10,
	  	'min_child_weight': 164,
	  	'gamma': 0.4,
	  	'sub_sample': 0.4,
	  	'colsample_bytree': 0.6,
     'alpha': 20, 
     'lamnbda': 0.6                                                
  }


  model = xgb.train(params, dtrain, 500, evals=wlist, verbose_eval=20, early_stopping_rounds=100)

  test_x = test[features]
  test_x = test_x[test_x['구분'] == i]
  test_X = xgb.DMatrix(test_x)

  preds = model.predict(test_X)
  pred.extend(preds)

  0%|          | 0/7 [00:00<?, ?it/s]

[0]	train-mae:1098.08	eval-mae:1153.93
Multiple eval metrics have been passed: 'eval-mae' will be used for early stopping.

Will train until eval-mae hasn't improved in 100 rounds.
[20]	train-mae:395.142	eval-mae:426.123
[40]	train-mae:148.03	eval-mae:170.154
[60]	train-mae:72.2306	eval-mae:107.258
[80]	train-mae:52.6899	eval-mae:94.033
[100]	train-mae:46.8188	eval-mae:91.5473
[120]	train-mae:43.4242	eval-mae:91.4228
[140]	train-mae:41.7118	eval-mae:90.8278
[160]	train-mae:40.6886	eval-mae:90.9747
[180]	train-mae:39.3555	eval-mae:91.6519
[200]	train-mae:38.5067	eval-mae:91.8471
[220]	train-mae:37.8729	eval-mae:92.5139
[240]	train-mae:37.1667	eval-mae:92.7942
Stopping. Best iteration:
[140]	train-mae:41.7118	eval-mae:90.8278



 14%|█▍        | 1/7 [00:16<01:39, 16.64s/it]

[0]	train-mae:900.56	eval-mae:953.333
Multiple eval metrics have been passed: 'eval-mae' will be used for early stopping.

Will train until eval-mae hasn't improved in 100 rounds.
[20]	train-mae:324.087	eval-mae:350.706
[40]	train-mae:121.878	eval-mae:142.493
[60]	train-mae:60.4943	eval-mae:93.0646
[80]	train-mae:44.7667	eval-mae:83.3178
[100]	train-mae:39.7498	eval-mae:81.432
[120]	train-mae:37.0846	eval-mae:81.3621
[140]	train-mae:35.8121	eval-mae:81.0738
[160]	train-mae:34.7842	eval-mae:81.3519
[180]	train-mae:33.9351	eval-mae:81.8831
[200]	train-mae:33.1781	eval-mae:82.2385
[220]	train-mae:32.4466	eval-mae:83.1021
[240]	train-mae:31.8623	eval-mae:83.7622
Stopping. Best iteration:
[141]	train-mae:35.6373	eval-mae:81.0393



 29%|██▊       | 2/7 [00:33<01:23, 16.71s/it]

...

 71%|███████▏  | 5/7 [01:24<00:35, 17.99s/it]

[0]	train-mae:1926.1	eval-mae:2168.17
Multiple eval metrics have been passed: 'eval-mae' will be used for early stopping.

Will train until eval-mae hasn't improved in 100 rounds.
[20]	train-mae:692.863	eval-mae:886.959
[40]	train-mae:257.854	eval-mae:399.729
[60]	train-mae:119.742	eval-mae:245.29
[80]	train-mae:83.1515	eval-mae:198.616
[100]	train-mae:72.3353	eval-mae:182.552
[120]	train-mae:67.4263	eval-mae:178.341
[140]	train-mae:64.5236	eval-mae:177.226
[160]	train-mae:62.1468	eval-mae:176.658
[180]	train-mae:59.7855	eval-mae:176.828
[200]	train-mae:57.5934	eval-mae:177.233
[220]	train-mae:55.9486	eval-mae:177.333
[240]	train-mae:54.992	eval-mae:178.091
Stopping. Best iteration:
[151]	train-mae:63.089	eval-mae:176.295



 86%|████████▌ | 6/7 [01:41<00:17, 17.82s/it]

[0]	train-mae:233.588	eval-mae:245.198
Multiple eval metrics have been passed: 'eval-mae' will be used for early stopping.

Will train until eval-mae hasn't improved in 100 rounds.
[20]	train-mae:84.1026	eval-mae:90.0584
[40]	train-mae:31.7137	eval-mae:35.9235
[60]	train-mae:15.8733	eval-mae:23.0939
[80]	train-mae:11.8501	eval-mae:20.4918
[100]	train-mae:10.5972	eval-mae:19.8898
[120]	train-mae:9.82037	eval-mae:19.8484
[140]	train-mae:9.35989	eval-mae:19.8872
[160]	train-mae:9.15432	eval-mae:19.8495
[180]	train-mae:8.87303	eval-mae:19.8435
[200]	train-mae:8.67394	eval-mae:20.0167
[220]	train-mae:8.52035	eval-mae:20.1105
[240]	train-mae:8.37245	eval-mae:20.1776
[260]	train-mae:8.27509	eval-mae:20.2075
Stopping. Best iteration:
[179]	train-mae:8.89811	eval-mae:19.8272



100%|██████████| 7/7 [02:00<00:00, 17.22s/it]


In [ ]:
submission_XGB_2 = submission
submission_XGB_2['공급량'] = pred

### 4) 앙상블


*   전체 데이터로 학습한 모델을 사용한 예측값과 구분별로 학습한 후 구분별로 예측한 값 0.6:0.4로 앙상블


In [ ]:
pred = submission_XGB_1['공급량'].values * 0.6 + submission_XGB_2['공급량'].values * 0.4

In [ ]:
submission_XGB = submission
submission_XGB['공급량'] = pred

In [ ]:
submission_XGB

,일자|시간|구분,공급량
0,2019-01-01 01 A,1887.324831
1,2019-01-01 02 A,1757.747971
2,2019-01-01 03 A,1612.638832
3,2019-01-01 04 A,1612.638832
4,2019-01-01 05 A,1753.136753
...,...,...
15115,2019-03-31 20 H,337.906781
15116,2019-03-31 21 H,347.392219
15117,2019-03-31 22 H,325.120437
15118,2019-03-31 23 H,297.854326


## 모델2: LGBM & XGBoost

###1) 학습

#####1. LGBM

In [ ]:
train_lgbm = train[features]
val_lgbm = val[features]

In [ ]:
# 파라미터 !!! 
params = {
    'objective': 'regression',
'boosting_type': ['gbdt'],
 'learning_rate': 0.02,
 'max_depth': 4,
 'metric': 'mae',
 'n_estimators': 10000,
 'random_state': 42,
'num_leaves': 506, 'min_child_samples': 99, 'reg_alpha': 0.1585234458533463, 'reg_lambda': 0.6535395893689313, 'colsample_bytree': 1.0, 'subsample': 0.4}



In [ ]:
ds_train = lgb.Dataset(train_x, train_y)
ds_val = lgb.Dataset(val_x, val_y)


lgbm_model = lgb.train(params, ds_train, 10000, ds_val, early_stopping_rounds=100)

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning:

Found `n_estimators` in params. Will use it instead of argument



[1]	valid_0's l1: 145.222
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's l1: 142.495
[3]	valid_0's l1: 139.842
[4]	valid_0's l1: 137.253
[5]	valid_0's l1: 134.685
[6]	valid_0's l1: 132.185
[7]	valid_0's l1: 129.72
[8]	valid_0's l1: 127.319
[9]	valid_0's l1: 124.948
[10]	valid_0's l1: 122.655
[11]	valid_0's l1: 120.38
[12]	valid_0's l1: 118.152
[13]	valid_0's l1: 115.926
[14]	valid_0's l1: 113.788
[15]	valid_0's l1: 111.723
[16]	valid_0's l1: 109.676
[17]	valid_0's l1: 107.702
[18]	valid_0's l1: 105.701
[19]	valid_0's l1: 103.718
[20]	valid_0's l1: 101.794
[21]	valid_0's l1: 99.9334
[22]	valid_0's l1: 98.0893
[23]	valid_0's l1: 96.2837
[24]	valid_0's l1: 94.5503
[25]	valid_0's l1: 92.8187
[26]	valid_0's l1: 91.1
[27]	valid_0's l1: 89.4335
[28]	valid_0's l1: 87.7921
[29]	valid_0's l1: 86.2542
[30]	valid_0's l1: 84.69
[31]	valid_0's l1: 83.1487
[32]	valid_0's l1: 81.6947
[33]	valid_0's l1: 80.2796
[34]	valid_0's l1: 78.8118
[35]	valid_0's l1: 77.4813
[36]	val

In [ ]:
y_val_lgbm_pred = lgbm_model.predict(val_lgbm)

#####2 XGBOOST

In [ ]:
xgb_train = train[features]
xgb_val = val[features]

In [ ]:
params = {
    'learning_rate': 0.02,
    'objective': 'reg:squarederror',
    'eval_metric':'mae', 
    'seed':42
}

In [ ]:
import xgboost as xgb

d_train = xgb.DMatrix(data=train_x, label = train_y)
d_val = xgb.DMatrix(data=val_x, label = val_y)
wlist = [(d_train, 'train'), (d_val,'eval')]

xgb_model = xgb.train(params, d_train, 1000, evals=wlist, verbose_eval=100, early_stopping_rounds=100)

[0]	train-mae:240.939	eval-mae:252.848
Multiple eval metrics have been passed: 'eval-mae' will be used for early stopping.

Will train until eval-mae hasn't improved in 100 rounds.
[100]	train-mae:33.7125	eval-mae:39.0868
[200]	train-mae:13.175	eval-mae:21.2767
[300]	train-mae:11.2108	eval-mae:20.1422
[400]	train-mae:10.4512	eval-mae:20.1717
Stopping. Best iteration:
[364]	train-mae:10.7191	eval-mae:20.1035



####2) 테스트 셋- 앙상블

In [ ]:
test_x = test[features]

In [ ]:
ensem_lgbm_preds = (lgbm_model.predict(test_x))

In [ ]:
xgb_test_x = xgb.DMatrix(test_x)
ensem_xgb_preds = (xgb_model.predict(xgb_test_x))

In [ ]:
pred2 =  ensem_lgbm_preds *0.6+ ensem_xgb_preds * 0.4

In [ ]:
submission_LGBM = submission
submission_LGBM['공급량'] = pred2

In [ ]:
submission_LGBM

,일자|시간|구분,공급량
0,2019-01-01 01 A,1887.324831
1,2019-01-01 02 A,1757.747971
2,2019-01-01 03 A,1612.638832
3,2019-01-01 04 A,1612.638832
4,2019-01-01 05 A,1753.136753
...,...,...
15115,2019-03-31 20 H,337.906781
15116,2019-03-31 21 H,347.392219
15117,2019-03-31 22 H,325.120437
15118,2019-03-31 23 H,297.854326


## 모델3: ExtraTree Regressor

### 1) 하이퍼파라미터 튜닝

하이퍼파라미터 튜닝을 위해 함수 설정

In [ ]:
def modeling_uncustomized (algorithm, x_train, y_train, x_test, y_test): # 하이퍼파라미터 조정 없이 모델 학습 uncustomized = algorithm(random_state=1234) uncustomized.fit(x_train, y_train) # Train Data 설명력 train_score_before = uncustomized.score(x_train, y_train).round(3) print(f"학습 데이터셋 정확도: {train_score_before}") # Test Data 설명력 test_score_before = uncustomized.score(x_test, y_test).round(3) print(f"테스트 데이터셋 정확도: {test_score_before}") return train_score_before, test_score_before
    uncustomized = algorithm(random_state=1234)
    uncustomized.fit(x_train, y_train)
    # Train Data 설명력
    train_score_before = uncustomized.score(x_train, y_train).round(3)
    print(f"학습 데이터셋 정확도: {train_score_before}")
    # Test Data 설명력
    test_score_before = uncustomized.score(x_test, y_test).round(3)
    print(f"테스트 데이터셋 정확도: {test_score_before}")
    return train_score_before, test_score_before

In [ ]:
def optimi_visualization(algorithm_name, x_values, train_score, test_score, xlabel, filename):
    plt.plot(x_values, train_score, linestyle = '-', label = 'train score')
    plt.plot(x_values, test_score, linestyle = '--', label = 'test score')
    plt.ylabel('Accuracy(%)')
    plt.xlabel(xlabel)
    plt.legend()

In [ ]:
def optimi_estimator(algorithm, algorithm_name, x_train, y_train, x_test, y_test, n_estimator_min, n_estimator_max):
    train_score = []; test_score =[]
    para_n_tree = [n_tree*5 for n_tree in range(n_estimator_min, n_estimator_max)]
    
    for v_n_estimators in para_n_tree:
        model = algorithm(n_estimators = v_n_estimators, random_state=1234)
        model.fit(x_train, y_train)
        train_score.append(model.score(x_train, y_train))
        test_score.append(model.score(x_test, y_test))
        
    df_score_n = pd.DataFrame({'n_estimators': para_n_tree, 'TrainScore': train_score, 'TestScore': test_score})
    optimi_visualization(algorithm_name, para_n_tree, train_score, test_score, "The number of estimator", "n_estimator")
    print(round(df_score_n, 4))

In [ ]:
def optimi_maxdepth (algorithm, algorithm_name, x_train, y_train, x_test, y_test, depth_min, depth_max, n_estimator):
    train_score = []; test_score = []
    para_depth = [depth for depth in range(depth_min, depth_max)]
    
    for v_max_depth in para_depth:
        if algorithm == DTC:
            model = algorithm(max_depth = v_max_depth,
                              random_state=1234)
        else:
            model = algorithm(max_depth = v_max_depth,
                              n_estimators = n_estimator,
                              random_state=1234)
            model.fit(x_train, y_train)
            train_score.append(model.score(x_train, y_train))
            test_score.append(model.score(x_test, y_test))
    df_score_n = pd.DataFrame({'depth': para_depth, 'TrainScore': train_score, 'TestScore': test_score})
    optimi_visualization(algorithm_name, para_depth, train_score, test_score, "The number of depth", "n_depth")
    print(round(df_score_n, 4))

In [ ]:
def optimi_minsplit (algorithm, algorithm_name, x_train, y_train, x_test, y_test, n_split_min, n_split_max, n_estimator, n_depth):
    train_score = []; test_score = []
    para_split = [n_split*2 for n_split in range(n_split_min, n_split_max)]
    for v_min_samples_split in para_split:
        if algorithm == DTC:
            model = algorithm(min_samples_split = v_min_samples_split,
                              max_depth = n_depth,
                              random_state = 1234)
        else:
            model = algorithm(min_samples_split = v_min_samples_split,
                              n_estimators = n_estimator,
                              max_depth = n_depth,
                              random_state = 1234)
        model.fit(x_train, y_train)
        train_score.append(model.score(x_train, y_train))
        test_score.append(model.score(x_test, y_test))
        
    df_score_n = pd.DataFrame({'min_samples_split': para_split, 'TrainScore': train_score, 'TestScore': test_score})
    optimi_visualization(algorithm_name, para_split, train_score, test_score, "The minimum number of samples required to split an internal node", "min_samples_split")
    print(round(df_score_n, 4))

In [ ]:
def optimi_minleaf(algorithm, algorithm_name, x_train, y_train, x_test, y_test, n_leaf_min, n_leaf_max, n_estimator, n_depth, n_split):
    train_score = []; test_score = []
    para_leaf = [n_leaf*2 for n_leaf in range(n_leaf_min, n_leaf_max)]
    
    for v_min_samples_leaf in para_leaf:
        if algorithm == DTC:
            model = algorithm(min_samples_leaf = v_min_samples_leaf,
                              max_depth = n_depth,
                              min_samples_split = n_split,
                              random_state=1234)
        else:
            model = algorithm(min_samples_leaf = v_min_samples_leaf,
                              n_estimators = n_estimator,
                              max_depth = n_depth,
                              min_samples_split = n_split,
                              random_state=1234)
        model.fit(x_train, y_train)
        train_score.append(model.score(x_train, y_train))
        test_score.append(model.score(x_test, y_test))
        
    df_score_n = pd.DataFrame({'min_samples_leaf': para_leaf, 'TrainScore': train_score, 'TestScore': test_score})
    optimi_visualization(algorithm_name, para_leaf, train_score, test_score, "The minimum number of samples required to be at a leaf node", "min_samples_leaf")
    print(round(df_score_n, 4))

In [ ]:
algorithm = ExtraTreesRegressor

하이퍼파라미터 튜닝 없이 기본 모델 학습

In [ ]:
train_acc_before, test_acc_before = modeling_uncustomized(algorithm,
                                                          train_x,
                                                          train_y,
                                                          val_x,
                                                          val_y)

**1단계 : n_estimators : 학습트리 개수 선정**

In [ ]:
n_estimator_min = 1
n_estimator_max = 30
optimi_estimator(algorithm, algorithm_name,
                 train_x, train_y, val_x, val_y,
                 n_estimator_min, n_estimator_max)

In [ ]:
n_estimator = 30

**2단계 : max_depth : 최대 깊이 선정**

In [ ]:
depth_min = 1
depth_max = 23
optimi_maxdepth(algorithm, algorithm_name,
                train_x, train_y, val_x, val_y,
                depth_min, depth_max, n_estimator)

In [ ]:
n_depth = 11

**3단계 : n_split : 분리 노드 최소 자료수 선정**

In [ ]:
n_split_min = 1
n_split_max = 101
pd.set_option('display.max_row', 100)
optimi_minsplit (algorithm, algorithm_name,
                 train_x, train_y, val_x, val_y,
                 n_split_min, n_split_max, n_estimator, n_depth)

In [ ]:
n_split = 32

**4단계 : n_leaf_nodes : 잎사귀 노드 최소 자료**

In [ ]:
n_leaf_min = 1
n_leaf_max = 101
optimi_minleaf(algorithm, algorithm_name,
               train_x, train_y, val_x, val_y,
               n_leaf_min, n_leaf_max, n_estimator, n_depth, n_split)

In [ ]:
n_leaf = 10

### 2) 학습

In [ ]:
total_x = total[features]
total_y = total['공급량']

In [ ]:
extra_model = ExtraTreesRegressor(random_state = 42,
                            n_estimators = 30,
                            min_samples_leaf = 10,
                            min_samples_split = 32,
                            max_depth = 11) 

In [ ]:
extra_model.fit(total_x, total_y)

ExtraTreesRegressor(max_depth=11, min_samples_leaf=10, min_samples_split=32,
                    n_estimators=30, random_state=42)

### 3) 예측

In [ ]:
test_x = test[features]

In [ ]:
extra_preds = extra_model.predict(test_x)

In [ ]:
submission_ET = submission
submission_ET['공급량'] = extra_preds

In [ ]:
submission_ET

,일자|시간|구분,공급량
0,2019-01-01 01 A,1887.324831
1,2019-01-01 02 A,1757.747971
2,2019-01-01 03 A,1612.638832
3,2019-01-01 04 A,1612.638832
4,2019-01-01 05 A,1753.136753
...,...,...
15115,2019-03-31 20 H,337.906781
15116,2019-03-31 21 H,347.392219
15117,2019-03-31 22 H,325.120437
15118,2019-03-31 23 H,297.854326


# 5. 최종 예측

**최종 앙상블**

In [ ]:
pred = submission_XGB['공급량'].values * 0.5 + submission_LGBM['공급량'].values * 0.3 + submission_ET['공급량'].values * 0.2

In [ ]:
submission['공급량'] = pred

In [ ]:
submission

,일자|시간|구분,공급량
0,2019-01-01 01 A,1887.324831
1,2019-01-01 02 A,1757.747971
2,2019-01-01 03 A,1612.638832
3,2019-01-01 04 A,1612.638832
4,2019-01-01 05 A,1753.136753
...,...,...
15115,2019-03-31 20 H,337.906781
15116,2019-03-31 21 H,347.392219
15117,2019-03-31 22 H,325.120437
15118,2019-03-31 23 H,297.854326


In [ ]:
submission.to_csv('쿠스트라다무스_최종.csv', index=False)